In [82]:
import sys
root_path = "/Users/sa/CS/stanford/cs229/Project/Neural-Response-Generator-augmented-with-psychological-personalities/"

import pandas as pd
import autoreload
import sys
sys.path.append(root_path + "library/")

%load_ext autoreload
%autoreload 2

# essentials
import numpy as np
from tqdm.notebook import tqdm
import pickle
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [207]:
script = pd.read_csv(root_path + '/data/intermediate/movie_script_elong.csv')

In [208]:
script.head()

,Unnamed: 0,char1Id,char2Id,movieId,lines
0,1716,u228,u233,m15,"[""This floor's freezing."", 'Christ. I never s..."
1,1717,u228,u233,m15,"[""Whoooah! No shit? I'm impressed."", ""Let's ..."
2,1718,u228,u233,m15,"[""Hey, 'Top.' What's the op?"", ""Rescue missio..."
3,1719,u228,u233,m15,"['Movement!', 'Position?', ""Can't lock up..."",..."
4,1720,u228,u231,m15,"[""...that's better. Pan it around a bit."", ""A..."


In [209]:
import ast

#contexted = []
groupid = 0
script_df = pd.DataFrame(columns=["groupId", "line"])
for itr, row in tqdm(script.iterrows(), total=script.shape[0], mininterval=20):
    for line in ast.literal_eval(row.lines):
        script_df.loc[script_df.shape[0]+1] = [groupid, line]
    groupid += 1
    #contexted.append(ast.literal_eval(row.lines))

In [210]:
script_df = script_df.reset_index(drop=True).copy()

In [211]:
script_df

,groupId,line
0,0,This floor's freezing.
1,0,Christ. I never saw such a buncha old women. ...
2,0,"Would you, Sir?"
3,1,Whoooah! No shit? I'm impressed.
4,1,Let's go...let's go. Cycle through!
...,...,...
27509,7066,How soon can it be altered?
27510,7066,Well...
27511,7066,Can we have it by tonight?
27512,7066,"Well, if it's absolutely necessary..."


# Prepare the data in conversational style with context

In [158]:
# load personality classifier model

from joblib import load

name = 'mlpc'
clf = load(root_path + 'models/final/' + name +'.joblib') 

import pickle
cvectorizer = pickle.load(open(root_path + "models/final/cvectorizer.pkl", "rb"))
vectorizer = pickle.load(open(root_path + "models/final/vectorizer.pkl", "rb"))
svd = pickle.load(open(root_path + "models/final/tsvd.pkl", "rb"))
label_dict = pickle.load(open(root_path + "models/final/label_dict.pkl", "rb"))

In [212]:
contexted = []

n = 7

for i in range(n, len(script_df['line'])):
    row = []
    prev = i - 1 - n
    c_grp = script_df['groupId'][i]
    for j in range(i, prev, -1):
        if script_df['groupId'][j] != c_grp:
            break
        row.append(script_df['line'][j])
    if len(row) == n+1:
        contexted.append(row) 

In [331]:
columns = ['response', 'context'] 
columns = columns + ['context/'+str(i) for i in range(n-1)]

In [214]:
df = pd.DataFrame.from_records(contexted, columns=columns)
df.head(5)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,Hard to say. It may have been blind instinct....,Could it be intelligent?,"Well, the queen is always physically larger th...",You're saying one of those things lays all the...,That is the question of the hour. We could as...,But these things come from eggs...so where are...,Yes. That follows.,"Which would mean lots of those parasites, righ..."
1,"If you go. It's a second chance, kiddo. And ...",If I go.,What if I said I could get you reinstated as a...,"It's all I could get. Anyway, it keeps my min...","Running loaders, forklifts, that sort of thing?",That's right.,I heard you were working in the cargo docks.,"Yeah, yeah. I saw the commercial."
2,"Spare me, Burke. I've had my psych evaluation...","If you go. It's a second chance, kiddo. And ...",If I go.,What if I said I could get you reinstated as a...,"It's all I could get. Anyway, it keeps my min...","Running loaders, forklifts, that sort of thing?",That's right.,I heard you were working in the cargo docks.
3,You mean dead.,Gone.,Where is she?,Yes. A little girl.,Did you ever have a baby?,"No, it's different, honey.",Isn't that how babies come? I mean people babi...,"I don't know, Newt. That's the truth."
4,Yes.,And he's been commissioned to write an opera. ...,Oh? News travels fast in Vienna.,"Well, I heard you met Herr Mozart.",Really? What else did he tell you today? Give ...,Well? How do you like it? It's Turkish. My hai...,Good morning.,Maestro.


In [216]:
personality = []

X_vec = vectorizer.transform(cvectorizer.transform(list(df.response)))
X_vec_svd = svd.transform(X_vec)
personality = np.asarray([label_dict[i] for i in clf.predict(X_vec_svd)])

In [219]:
from sklearn.model_selection import train_test_split

trn_df, val_df = train_test_split(df.loc[personality == 'ISFJ'], test_size = 0.1)
#trn_df, val_df = train_test_split(df, test_size = 0.1) # fullset training
print (trn_df.shape)
trn_df.head()

(685, 8)


,response,context,context/0,context/1,context/2,context/3,context/4,context/5
2516,How can we? We're broke.,"You promised - the minute we hit Florida, we w...",Blow where?,I'm tired of being a flag. I want to be a bul...,They don't care - just as long as you wear ski...,And I'm not even pretty.,"Well, now you know how the other half lives.",I got pinched in the elevator.
1467,"Gee, Lou, I'm doing the best I can.","If she's not there, you're going downtown, and...",What for?,"No, Gittes, you'll show us.",She lives in Pedro. I'll write it down for you.,What's the maid's address?,Her maid's house. I think she knows something'...,Where?
228,Myself. I found myself much more reasonable.,Who did you bribe for your visa? Renault or yo...,"Well, Rick, after tonight I'll be through with...",I don't mind a parasite. I object to a cut-rat...,But think of all the poor devils who cannot me...,"For a price, Ugarte, for a price.","But why? Oh, you object to the kind of busines...","If I gave you any thought, I probably would."
2579,That's it. Something tells me she's going to ...,Do you suppose she went out shopping?,Yeah. This is one for Ripley.,Believe it or not - Josephine predicted the wh...,"No, no, Sugar - wait. I have a feeling she's ...","Well, I'll come back later.",That's funny. Josie - I can't imagine where ...,I guess she's not in here.
2822,"Hellnno, Major. I was just trying to think th...",Are you saying our order don't make sense?,"Well, if they hit us first, they've probably f...","What's your point, Lothar?","Well, if there's a war, they must have hit us ...",That's what I said.,Our targets are a missile complex and an airfi...,"Shoot, Lothar."


In [334]:
trn_df.to_csv(root_path + "data/processed/final/trn_df.csv")
val_df.to_csv(root_path + "data/processed/final/val_df.csv")